In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
df = pd.read_csv('/content/Reviews only (Sentiment) only 20.csv')
df.head()

Break Reviews in to features

In [ ]:
def breakSentences(word_array):
  sentence_all = []
  for index, row in df.iterrows():
    sentences_with_feature = []
    for sen in sent_tokenize(row['reviewText']):
        l = word_tokenize(sen)
        if len(set(l).intersection(word_array))>0:
            sentences_with_feature.append(sen)
    sentence_all.append(sentences_with_feature)
  list_of_tuples = list(zip(sentence_all)) 
  df_sentences = pd.DataFrame(list_of_tuples,columns = ['broken'])
  return df_sentences

In [ ]:
# Audio_quality = ["sound","quality"]
Audio_quality = ["sound","quality","immersive"]
sentences_with_Audio=breakSentences(Audio_quality)
sentences_with_Audio

In [ ]:
Battery_quality = ["long lasting","battery","but"]
sentences_with_Battery=breakSentences(Battery_quality)
sentences_with_Battery

In [ ]:
build_quality = ["fit","feel","comfort","soft"]
sentences_with_build=breakSentences(build_quality)
sentences_with_build

add sentiment labels 

> Valence Aware Dictionary for Sentiment Reasoning (VADER) is a model used for text 

> sentiment analysis its sensitve for both polarity

In [ ]:
# importing Sentiment Intensity Analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#SentimentIntensityAnalyzer() returns dictanry Scores of negative, positive, netural, compound
sid = SentimentIntensityAnalyzer()

In [ ]:
def sentiment(sentenceArray,feature):
  df_audio = sentenceArray
  # df_audio.insert(1, "lab",0)

  for index, row in df_audio.iterrows():
    list_of_tuples = list(zip(row['broken'])) 
    df_audio_sentences = pd.DataFrame(list_of_tuples,columns = [feature+'_sen'])

    df_audio_sentences['scores'] = df_audio_sentences[feature+'_sen'].apply(lambda audio_sen: sid.polarity_scores(str(audio_sen)))
    df_audio_sentences['compound']  = df_audio_sentences['scores'].apply(lambda score_dict: score_dict['compound'])
    df_audio_sentences[feature+'_label'] = df_audio_sentences['compound'].apply(lambda c: 'pos' if c >=0.5 else ('neu' if c >= -0.5 else 'neg'))
    
    total = df_audio_sentences['compound'].sum()
    numRow = df_audio_sentences['compound'].count()
    averageCompound = float(total/numRow)

    if np.isnan(averageCompound):
      sen = "NA"
    elif averageCompound >= 0:
      sen = "pos"
    elif (averageCompound > -0.25) and (averageCompound < 0):
      sen = "nue"
    elif averageCompound <= -0.25:
      sen = "neg"

    df_audio.loc[index, feature+'_label'] = sen
  # df_audio
  # df_audio_sentences
  return df_audio.head(100)

In [ ]:
sentiment(sentences_with_Audio,"Audio")

In [ ]:
sentiment(sentences_with_Battery,"Battery")

In [ ]:
sentiment(sentences_with_build,"Build")